In [1]:
!pip install textstat

     |████████████████████████████████| 101 kB 6.1 MB/s 
     |████████████████████████████████| 2.0 MB 62.4 MB/s 


In [2]:
!pip install -U spacy

     |████████████████████████████████| 6.0 MB 25.0 MB/s 
     |████████████████████████████████| 451 kB 58.1 MB/s 
     |████████████████████████████████| 181 kB 63.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 10.1 MB 48.3 MB/s 
     |████████████████████████████████| 628 kB 57.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [3]:
!pip install nltk

In [4]:
import numpy as np
import pandas as pd
import string
import re
import warnings

import textstat
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import spacy

warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%cd /content/drive/MyDrive/gitrepo/probspace_kiva

/content/drive/MyDrive/gitrepo/probspace_kiva


In [12]:
train_df = pd.read_csv('/content/drive/MyDrive/gitrepo/probspace_kiva/input/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/gitrepo/probspace_kiva/input/test.csv')

## DESCRIPTION_TRANSLATEDの欠損値の置換

In [13]:
train_df.loc[train_df['DESCRIPTION_TRANSLATED'].isna(), 'DESCRIPTION_TRANSLATED'] = train_df.loc[train_df['DESCRIPTION_TRANSLATED'].isna(), 'DESCRIPTION']

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91333 entries, 0 to 91332
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   LOAN_ID                          91333 non-null  int64  
 1   ORIGINAL_LANGUAGE                91333 non-null  object 
 2   DESCRIPTION                      91333 non-null  object 
 3   DESCRIPTION_TRANSLATED           91333 non-null  object 
 4   LOAN_AMOUNT                      91333 non-null  int64  
 5   IMAGE_ID                         91333 non-null  int64  
 6   ACTIVITY_NAME                    91333 non-null  object 
 7   SECTOR_NAME                      91333 non-null  object 
 8   LOAN_USE                         91333 non-null  object 
 9   COUNTRY_CODE                     91333 non-null  object 
 10  COUNTRY_NAME                     91333 non-null  object 
 11  TOWN_NAME                        88573 non-null  object 
 12  CURRENCY_POLICY   

##  DESCRIPTIONとDESCRIPTION_TRANSLATEDの前処理

In [15]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]


html_tags = ['<p>', '</p>', '<table>', '</table>', '<tr>', '</tr>', '<ul>', '<ol>', '<dl>', '</ul>', '</ol>',
             '</dl>', '<li>', '<dd>', '<dt>', '</li>', '</dd>', '</dt>', '<h1>', '</h1>',
             '<br>', '<br/>', '<br />','<strong>', '</strong>', '<span>', '</span>', '<blockquote>', '</blockquote>',
             '<pre>', '</pre>', '<div>', '</div>', '<h2>', '</h2>', '<h3>', '</h3>', '<h4>', '</h4>', '<h5>', '</h5>',
             '<h6>', '</h6>', '<blck>', '<pr>', '<code>', '<th>', '</th>', '<td>', '</td>', '<em>', '</em>']

empty_expressions = ['&lt;', '&gt;', '&amp;', '&nbsp;', 
                     '&emsp;', '&ndash;', '&mdash;', '&ensp;'
                     '&quot;', '&#39;']

In [16]:
def pre_preprocess(x):
    return str(x).lower()

def rm_spaces(text):
    spaces = ['\u200b', '\u200e', '\u202a', '\u2009', '\u2028', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\u3000', '\x10', '\x7f', '\x9d', '\xad',
              '\x97', '\x9c', '\x8b', '\x81', '\x80', '\x8c', '\x85', '\x92', '\x88', '\x8d', '\x80', '\x8e', '\x9a', '\x94', '\xa0', 
              '\x8f', '\x82', '\x8a', '\x93', '\x90', '\x83', '\x96', '\x9b', '\x9e', '\x99', '\x87', '\x84', '\x9f',
             ]
    for space in spaces:
            text = text.replace(space, ' ')
    return text

def remove_urls(x):
    x = re.sub(r'(https?://[a-zA-Z0-9.-]*)', r'', x)

    # original
    x = re.sub(r'(quote=\w+\s?\w+;?\w+)', r'', x)
    return x

def clean_puncts(x):
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_html_tags(x, stop_words=[]):      
    for r in html_tags:
        x = x.replace(r, '')
    for r in empty_expressions:
        x = x.replace(r, ' ')
    for r in stop_words:
        x = x.replace(r, '')
    return x

def preprocess(data):
    data = data.apply(lambda x: pre_preprocess(x))
    data = data.apply(lambda x: rm_spaces(x))
    data = data.apply(lambda x: remove_urls(x))
    data = data.apply(lambda x: clean_puncts(x))
    data = data.apply(lambda x: clean_html_tags(x))
    return data

In [17]:
train_df['clean_DESCRIPTION_TRANSLATED'] = preprocess(train_df['DESCRIPTION_TRANSLATED'])
test_df['clean_DESCRIPTION_TRANSLATED'] = preprocess(test_df['DESCRIPTION_TRANSLATED'])

In [18]:
train_df.head()

,LOAN_ID,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,IMAGE_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,COUNTRY_NAME,TOWN_NAME,CURRENCY_POLICY,CURRENCY_EXCHANGE_COVERAGE_RATE,CURRENCY,TAGS,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL,clean_DESCRIPTION_TRANSLATED
0,1733169,English,Teodora is a 50-year-old married woman from th...,Teodora is a 50-year-old married woman from th...,100,3115271,Weaving,Arts,"to purchase materials like nipa palm, bamboo ...",PH,Philippines,"Maribojoc, Bohol",shared,0.1,PHP,#Elderly,monthly,field_partner,teodora is a 50 - year - old married woman fro...
1,1546998,English,Diego is 32 years old and lives in the municip...,Diego is 32 years old and lives in the municip...,1350,2870403,Barber Shop,Services,"to buy two hair clippers, a new barber chair, ...",CO,Colombia,Apartadó,shared,0.1,COP,"user_favorite, user_favorite",monthly,field_partner,diego is 32 years old and lives in the municip...
2,1808517,Spanish,"Osman, es un joven de 27 años de edad, soltero...","Osman is a young man, 27 years old, single, an...",225,3215705,Farming,Agriculture,to purchase sacks of fertilizers to care for a...,HN,Honduras,"Nueva Frontera, Santa Barbara.",shared,0.1,HNL,NaN,bullet,field_partner,"osman is a young man , 27 years old , single..."
3,1452940,English,"His name is Nino, 31 years old, married to Che...","His name is Nino, 31 years old, married to Che...",350,2745031,Motorcycle Transport,Transportation,"to pay for fuel, tires and change oil for his ...",PH,Philippines,"Silang, Cavite",shared,0.1,PHP,user_favorite,monthly,field_partner,"his name is nino , 31 years old , married to..."
4,1778420,English,"Pictured above is Teresa, often described as a...","Pictured above is Teresa, often described as a...",625,3083800,Farming,Agriculture,to purchase hybrid seeds and fertilizer to imp...,KE,Kenya,Mumias,shared,0.1,KES,"#Eco-friendly, #Sustainable Ag, #Parent, #Elde...",bullet,field_partner,"pictured above is teresa , often described as..."


## 言語特徴量の抽出

### カウントベース

In [19]:
import nltk
nltk.download('punkt')

def create_basic_feature(df, target_columns):
    # スペースのぞいた文字数
    df['char_count'] = [len(t.replace(' ', '')) for t in df[target_columns]]
    
    # 単語
    df['word_count'] = [len(str(t).split()) for t in df[target_columns]] 
    df['word_unique_count'] = [len(set(str(t).split())) for t in df[target_columns]]
    df['word_unique_ratio'] = df['word_unique_count'] / (df['word_count'] + 1)
    df['word_ave_length'] = [sum([len(w) for w in t.split()])/len(t.split()) for t in df[target_columns]] 
    
    # 句読点
    punctuations = string.punctuation
    df['punc_count'] = [len([w for w in t if w in punctuations]) for t in df[target_columns]]
    
    # 大文字、小文字
    df['uppercase'] = df[target_columns].str.findall(r'[A-Z]').str.len()+1
    df['lowercase'] = df[target_columns].str.findall(r'[a-z]').str.len()+1
    df['up_low_ratio'] = df['uppercase'] / (df['lowercase'] + 1)
    
    # 段落
    df['paragraph_count'] = [t.count('\n') for t in df[target_columns]]

    # 文章
    df['sentence_count'] = [len(sent_tokenize(t)) for t in df[target_columns]]
    df['sentence_ave_length'] = [sum([len(s) for s in sent_tokenize(t)])/len(sent_tokenize(t)) for t in df[target_columns]] 
    df['sentence_max_length'] = [max([len(s) for s in sent_tokenize(t)]) for t in df[target_columns]]
    df['sentence_max_length'] = [min([len(s) for s in sent_tokenize(t)]) for t in df[target_columns]]                                                                                                               
    df['word_per_sentence'] = df['word_count'] / df['sentence_count']

    # 母音
    df['syllable_count'] = [textstat.syllable_count(t) for t in df[target_columns]]
    df['syllable_per_sentence'] = df['syllable_count'] / df['sentence_count']
    df['syllable_per_word'] = df['syllable_count'] / df['word_count']
    
    return df

train_df = create_basic_feature(train_df, target_columns='DESCRIPTION_TRANSLATED')
test_df = create_basic_feature(test_df, target_columns='DESCRIPTION_TRANSLATED')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Vectorization

In [20]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.4 MB 5.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [21]:
def spacy_features(df: pd.DataFrame, target_columns: str):
    nlp = spacy.load('en_core_web_lg')
    with nlp.disable_pipes():
        vectors = np.array([nlp(text).vector for text in df[target_columns]])
    return vectors

train_text_vector = pd.DataFrame(spacy_features(train_df, target_columns='clean_DESCRIPTION_TRANSLATED'), columns=['vec_'+str(n) for n in range(300)])
test_text_vector = pd.DataFrame(spacy_features(test_df, target_columns='clean_DESCRIPTION_TRANSLATED'), columns=['vec_'+str(n) for n in range(300)])

train_df = pd.concat([train_df, train_text_vector], axis=1)
test_df = pd.concat([test_df, test_text_vector], axis=1)

In [22]:
train_df.head()

,LOAN_ID,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,IMAGE_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,COUNTRY_NAME,TOWN_NAME,CURRENCY_POLICY,CURRENCY_EXCHANGE_COVERAGE_RATE,CURRENCY,TAGS,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL,clean_DESCRIPTION_TRANSLATED,char_count,word_count,word_unique_count,word_unique_ratio,word_ave_length,punc_count,uppercase,lowercase,up_low_ratio,paragraph_count,sentence_count,sentence_ave_length,sentence_max_length,word_per_sentence,syllable_count,syllable_per_sentence,syllable_per_word,vec_0,vec_1,vec_2,vec_3,...,vec_260,vec_261,vec_262,vec_263,vec_264,vec_265,vec_266,vec_267,vec_268,vec_269,vec_270,vec_271,vec_272,vec_273,vec_274,vec_275,vec_276,vec_277,vec_278,vec_279,vec_280,vec_281,vec_282,vec_283,vec_284,vec_285,vec_286,vec_287,vec_288,vec_289,vec_290,vec_291,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299
0,1733169,English,Teodora is a 50-year-old married woman from th...,Teodora is a 50-year-old married woman from th...,100,3115271,Weaving,Arts,"to purchase materials like nipa palm, bamboo ...",PH,Philippines,"Maribojoc, Bohol",shared,0.1,PHP,#Elderly,monthly,field_partner,teodora is a 50 - year - old married woman fro...,600,122,91,0.739837,4.918033,45,13,538,0.024119,0,4,179.500000,66,30.500000,169,42.250000,1.385246,-0.008935,0.154591,-0.093974,-0.045663,...,0.038450,0.122658,0.054574,0.087442,0.046162,-0.154659,-0.069359,-0.052340,0.198813,-0.087857,-0.024616,-0.041476,-0.063640,-0.105062,-0.089810,-0.040226,0.032966,0.116964,-0.052537,0.100891,0.131845,0.034739,0.037437,-0.056944,0.018437,-0.066491,0.062150,-0.070571,0.120410,-0.020444,-0.082336,0.004650,-0.023203,-0.064205,-0.055478,0.034566,-0.010977,-0.092714,-0.035470,0.049837
1,1546998,English,Diego is 32 years old and lives in the municip...,Diego is 32 years old and lives in the municip...,1350,2870403,Barber Shop,Services,"to buy two hair clippers, a new barber chair, ...",CO,Colombia,Apartadó,shared,0.1,COP,"user_favorite, user_favorite",monthly,field_partner,diego is 32 years old and lives in the municip...,853,180,113,0.624309,4.738889,48,19,780,0.024328,0,11,92.909091,36,16.363636,268,24.363636,1.488889,-0.011556,0.179395,-0.113714,-0.057885,...,0.035360,0.172441,0.030317,0.060893,0.059494,-0.096022,-0.055573,-0.012669,0.188907,0.010173,-0.006245,-0.004830,-0.081646,-0.103878,-0.083226,-0.010761,0.028462,0.103360,0.038940,0.113650,0.107337,0.064731,-0.007046,0.000689,0.019605,-0.045657,0.103863,-0.076530,0.096132,-0.020631,-0.076051,-0.000768,-0.043774,-0.025919,0.038349,-0.007879,0.050327,-0.058147,-0.044473,0.058082
2,1808517,Spanish,"Osman, es un joven de 27 años de edad, soltero...","Osman is a young man, 27 years old, single, an...",225,3215705,Farming,Agriculture,to purchase sacks of fertilizers to care for a...,HN,Honduras,"Nueva Frontera, Santa Barbara.",shared,0.1,HNL,NaN,bullet,field_partner,"osman is a young man , 27 years old , single...",467,103,74,0.711538,4.533981,25,12,425,0.028169,0,6,94.000000,51,17.166667,146,24.333333,1.417476,-0.037868,0.168544,-0.083576,-0.095001,...,0.015766,0.145665,0.040804,0.073544,0.034250,-0.161097,-0.053630,-0.035294,0.137471,-0.064863,0.068038,-0.021864,-0.036440,-0.131327,-0.073344,-0.033604,-0.004774,0.130011,0.031723,0.116239,0.109728,0.037722,-0.035488,-0.032668,-0.007088,-0.048100,0.063344,-0.091688,0.091032,0.008629,-0.069563,0.025016,-0.009223,-0.054928,-0.033079,-0.024544,0.032128,-0.055155,-0.080539,0.048633
3,1452940,English,"His name is Nino, 31 years old, married to Che...","His name is Nino, 31 years old, married to Che...",350,2745031,Motorcycle Transport,Transportation,"to pay for fuel, tires and change oil for his ...",PH,Philippines,"Silang, Cavite",shared,0.1,PHP,user_favorite,monthly,field_partner,"his name is nino , 31 years old , married to...",1098,234,156,0.663830,4.692308,46,26,1022,0.025415,0,7,189.285714,62,33.428571,336,48.000000,1.435897,0.015259,0.127146,-0.096488,-0.050258,...,0.020326,0.137926,0.008104,0.073

### tf-idf

In [23]:
nltk.download('stopwords')
STOP_WORDS = set(stopwords.words("english"))

def preprocess_sentence_tfidf(df): 
    stem = PorterStemmer()
    word_list = []
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', df) #句読点削除
    tokenized_word = word_tokenize(text) #単語
    for word in tokenized_word:
        if word not in STOP_WORDS: #ストップワード削除
            word = stem.stem(word) #ステミング
            word_list.append(word) 
    return ' '.join(word_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
train_df['clean_DESCRIPTION_TRANSLATED_tfidf'] = train_df['clean_DESCRIPTION_TRANSLATED'].apply(lambda x : preprocess_sentence_tfidf(x))
test_df['clean_DESCRIPTION_TRANSLATED_tfidf'] = test_df['clean_DESCRIPTION_TRANSLATED'].apply(lambda x : preprocess_sentence_tfidf(x))

In [25]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
all_en_text_df = pd.concat([train_df['clean_DESCRIPTION_TRANSLATED_tfidf'], test_df['clean_DESCRIPTION_TRANSLATED_tfidf']]).reset_index(drop=True)
tfidf_vec = TfidfVectorizer(dtype=np.float32, sublinear_tf=True, use_idf=True, smooth_idf=True)

all_en_text_tfidf = tfidf_vec.fit_transform(all_en_text_df)

In [26]:
all_en_text_tfidf[[1, 2, 4]]

<3x63713 sparse matrix of type '<class 'numpy.float32'>'
	with 181 stored elements in Compressed Sparse Row format>

In [28]:
import joblib
joblib.dump(all_en_text_tfidf, 'output/preprocess/preprocess_v1/tfidf_en_text.joblib')

['/content/drive/MyDrive/gitrepo/probspace_kiva/preprocess/preprocess_v1/tfidf_en_text.joblib']

In [29]:
train_df.to_csv('/content/drive/MyDrive/gitrepo/probspace_kiva/preprocess/preprocess_v1/preprocess_train.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/gitrepo/probspace_kiva/preprocess/preprocess_v1/preprocess_test.csv', index=False)

In [30]:
all_en_text_tfidf[[1, 2, 4] + [5, 6, 9]]

<6x63713 sparse matrix of type '<class 'numpy.float32'>'
	with 309 stored elements in Compressed Sparse Row format>

In [31]:
all_en_text_tfidf[91333:]

<91822x63713 sparse matrix of type '<class 'numpy.float32'>'
	with 4432491 stored elements in Compressed Sparse Row format>

In [32]:
test_df.shape

(91822, 336)

In [33]:
train_df.shape

(91333, 337)